# Complete Historical Dataset Validation with Adaptive Learning

**Objective**: Run reverse prediction evaluation on complete historical dataset to test if physics model can learn and outperform heat curve over time.

**Key Features**:
- Chronological processing for proper adaptive learning
- Reverse prediction methodology throughout
- Learning progression tracking
- Head-to-head performance comparison

In [ ]:
# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import warnings
import os
from dotenv import load_dotenv
from tqdm.notebook import tqdm

warnings.filterwarnings('ignore')

# Load environment variables
load_dotenv()

# Import notebook helpers and enhanced model
from notebook_imports import create_influx_service

# Use the working RealisticPhysicsModel instead of ThermalEquilibriumModel
import sys
sys.path.append('../src')

try:
    from src.physics_model import RealisticPhysicsModel
    print("✅ Using RealisticPhysicsModel")
except ImportError:
    try:
        from physics_model import RealisticPhysicsModel
        print("✅ Using RealisticPhysicsModel (fallback)")
    except ImportError:
        print("❌ Could not import RealisticPhysicsModel")
        raise

print("🚀 Complete Historical Dataset Validation with Adaptive Learning")
print(f"📅 Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}")
print("✅ Reverse prediction methodology with full dataset processing")

In [ ]:
# Enhanced Heat Curve with Reverse Prediction and Performance Tracking
class HeatCurveWithReverse:
    """Heat curve baseline with reverse prediction capability."""
    
    def __init__(self):
        # Your proven heat curve parameters
        self.points = {"x1": -15.0, "y1": 64.0, "x2": 18.0, "y2": 31.0}
        self.slope = (self.points["y1"] - self.points["y2"]) / (self.points["x1"] - self.points["x2"])
        self.intercept = self.points["y2"] - (self.slope * self.points["x2"])
        
        # Performance tracking for reverse predictions
        self.reverse_predictions = []
        self.reverse_errors = []
        self.performance_history = []  # Rolling performance metrics
        
    def predict_outlet_temperature(self, current_indoor, target_indoor, outdoor_temp, 
                                 outdoor_forecast=None, shift_value=0):
        """Forward prediction: predict outlet temperature."""
        if outdoor_forecast is not None:
            target_temp = outdoor_temp * 0.6 + outdoor_forecast * 0.4
        else:
            target_temp = outdoor_temp
            
        outlet_temp = self.slope * target_temp + self.intercept + shift_value
        return max(16.0, min(65.0, outlet_temp))
    
    def reverse_predict_outlet_temperature(self, achieved_indoor, target_indoor, outdoor_temp,
                                         outdoor_forecast=None):
        """REVERSE prediction: Given achieved indoor temp, what outlet should we have predicted?"""
        # For heat curve, outlet depends mainly on outdoor temperature
        # But we can adjust based on how far off we were from target
        
        base_outlet = self.predict_outlet_temperature(achieved_indoor, target_indoor, outdoor_temp, outdoor_forecast)
        
        # Adjustment based on actual result vs target
        temp_error = achieved_indoor - target_indoor
        
        # If we overshot (positive error), we should have used lower outlet
        # If we undershot (negative error), we should have used higher outlet
        adjustment = -temp_error * 3.0  # 3°C outlet adjustment per 1°C indoor error
        
        reverse_predicted_outlet = base_outlet + adjustment
        return max(16.0, min(65.0, reverse_predicted_outlet))
    
    def track_reverse_prediction(self, reverse_predicted_outlet, actual_outlet, timestamp):
        """Track reverse prediction performance."""
        outlet_error = abs(reverse_predicted_outlet - actual_outlet)
        
        self.reverse_predictions.append({
            'timestamp': timestamp,
            'reverse_predicted_outlet': reverse_predicted_outlet,
            'actual_outlet': actual_outlet,
            'outlet_error': outlet_error
        })
        
        self.reverse_errors.append(outlet_error)
        
        # Update rolling performance every 100 predictions
        if len(self.reverse_errors) % 100 == 0:
            self._update_performance_history()
    
    def _update_performance_history(self):
        """Update rolling performance metrics."""
        if len(self.reverse_errors) >= 100:
            recent_errors = self.reverse_errors[-100:]  # Last 100 predictions
            self.performance_history.append({
                'prediction_count': len(self.reverse_errors),
                'avg_error': np.mean(recent_errors),
                'std_error': np.std(recent_errors),
                'within_5C': sum(1 for e in recent_errors if e <= 5.0) / len(recent_errors) * 100
            })
    
    def get_reverse_performance_metrics(self):
        """Get reverse prediction performance metrics."""
        if not self.reverse_errors:
            return {'insufficient_data': True}
            
        return {
            'avg_outlet_error': np.mean(self.reverse_errors),
            'max_outlet_error': np.max(self.reverse_errors),
            'std_outlet_error': np.std(self.reverse_errors),
            'total_predictions': len(self.reverse_predictions),
            'within_2C': sum(1 for e in self.reverse_errors if e <= 2.0) / len(self.reverse_errors) * 100,
            'within_5C': sum(1 for e in self.reverse_errors if e <= 5.0) / len(self.reverse_errors) * 100,
            'performance_history': self.performance_history
        }

In [ ]:
# Enhanced Physics Model with Reverse Prediction and Adaptive Learning Tracking
class PhysicsModelWithReverse:
    """Wrapper for ThermalEquilibriumModel with reverse prediction capability."""
    
    def __init__(self):
        self.model = ThermalEquilibriumModel()
        
        # Performance tracking for reverse predictions
        self.reverse_predictions = []
        self.reverse_errors = []
        self.performance_history = []  # Rolling performance metrics
        self.parameter_evolution = []  # Track how parameters evolve
        
    def calculate_optimal_outlet_temperature(self, *args, **kwargs):
        """Forward prediction: calculate optimal outlet temperature."""
        return self.model.calculate_optimal_outlet_temperature(*args, **kwargs)
    
    def predict_equilibrium_temperature(self, *args, **kwargs):
        """Forward prediction: predict equilibrium temperature."""
        return self.model.predict_equilibrium_temperature(*args, **kwargs)
        
    def update_prediction_feedback(self, *args, **kwargs):
        """Update adaptive learning."""
        result = self.model.update_prediction_feedback(*args, **kwargs)
        
        # Track parameter evolution
        if hasattr(self.model, 'calibration_tracker') and self.model.calibration_tracker:
            latest_params = self.model.calibration_tracker.get_current_parameters()
            self.parameter_evolution.append({
                'prediction_count': len(self.reverse_errors),
                'timestamp': kwargs.get('timestamp', datetime.now()),
                'parameters': latest_params.copy() if latest_params else None
            })
        
        return result
        
    def get_adaptive_learning_metrics(self):
        """Get adaptive learning metrics."""
        return self.model.get_adaptive_learning_metrics()
        
    def reset_adaptive_learning(self):
        """Reset adaptive learning state."""
        return self.model.reset_adaptive_learning()
    
    def reverse_predict_outlet_temperature(self, achieved_indoor, target_indoor, outdoor_temp,
                                         outdoor_forecast=None, pv_power=0):
        """REVERSE prediction: Given achieved indoor temp, what outlet should we have predicted?"""
        
        # Use physics model to reverse-engineer the outlet temperature
        # Binary search to find optimal outlet temperature
        low_outlet = 16.0
        high_outlet = 65.0
        tolerance = 0.1  # °C tolerance
        
        for _ in range(20):  # Maximum iterations
            mid_outlet = (low_outlet + high_outlet) / 2
            
            # Predict what indoor temp this outlet would achieve
            predicted_indoor = self.model.predict_equilibrium_temperature(
                mid_outlet, outdoor_temp, pv_power=pv_power
            )
            
            # Check if we're close enough to achieved indoor
            error = predicted_indoor - achieved_indoor
            
            if abs(error) < tolerance:
                return mid_outlet
            elif error > 0:  # Predicted too high, reduce outlet
                high_outlet = mid_outlet
            else:  # Predicted too low, increase outlet
                low_outlet = mid_outlet
                
        # Return best estimate if couldn't converge
        return (low_outlet + high_outlet) / 2
    
    def track_reverse_prediction(self, reverse_predicted_outlet, actual_outlet, timestamp):
        """Track reverse prediction performance."""
        outlet_error = abs(reverse_predicted_outlet - actual_outlet)
        
        self.reverse_predictions.append({
            'timestamp': timestamp,
            'reverse_predicted_outlet': reverse_predicted_outlet,
            'actual_outlet': actual_outlet,
            'outlet_error': outlet_error
        })
        
        self.reverse_errors.append(outlet_error)
        
        # Update rolling performance every 100 predictions
        if len(self.reverse_errors) % 100 == 0:
            self._update_performance_history()
    
    def _update_performance_history(self):
        """Update rolling performance metrics."""
        if len(self.reverse_errors) >= 100:
            recent_errors = self.reverse_errors[-100:]  # Last 100 predictions
            self.performance_history.append({
                'prediction_count': len(self.reverse_errors),
                'avg_error': np.mean(recent_errors),
                'std_error': np.std(recent_errors),
                'within_5C': sum(1 for e in recent_errors if e <= 5.0) / len(recent_errors) * 100
            })
    
    def get_reverse_performance_metrics(self):
        """Get reverse prediction performance metrics."""
        if not self.reverse_errors:
            return {'insufficient_data': True}
            
        return {
            'avg_outlet_error': np.mean(self.reverse_errors),
            'max_outlet_error': np.max(self.reverse_errors),
            'std_outlet_error': np.std(self.reverse_errors),
            'total_predictions': len(self.reverse_predictions),
            'within_2C': sum(1 for e in self.reverse_errors if e <= 2.0) / len(self.reverse_errors) * 100,
            'within_5C': sum(1 for e in self.reverse_errors if e <= 5.0) / len(self.reverse_errors) * 100,
            'performance_history': self.performance_history,
            'parameter_evolution': self.parameter_evolution
        }

In [ ]:
# Load historical data for validation
print("📊 Loading historical heating data...")

# Initialize InfluxDB service
influx_service = create_influx_service()

if influx_service is None:
    print("❌ Could not connect to InfluxDB. Please check your configuration.")
    raise ConnectionError("InfluxDB connection failed")

# Define time range for analysis (last 30 days for comprehensive testing)
end_time = datetime.now()
start_time = end_time - timedelta(days=30)

print(f"📅 Analysis period: {start_time.strftime('%Y-%m-%d')} to {end_time.strftime('%Y-%m-%d')}")

# Load heating system data
heating_data = influx_service.query_heating_data(start_time, end_time)

print(f"✅ Loaded {len(heating_data)} data points")
print(f"📈 Data columns: {list(heating_data.columns)}")

# Display basic statistics
if len(heating_data) > 0:
    print("\n📊 Data Overview:")
    print(f"   • Indoor temp range: {heating_data['indoor_temperature'].min():.1f}°C to {heating_data['indoor_temperature'].max():.1f}°C")
    print(f"   • Outdoor temp range: {heating_data['outdoor_temperature'].min():.1f}°C to {heating_data['outdoor_temperature'].max():.1f}°C")
    if 'outlet_temperature' in heating_data.columns:
        print(f"   • Outlet temp range: {heating_data['outlet_temperature'].min():.1f}°C to {heating_data['outlet_temperature'].max():.1f}°C")
    print(f"   • Time span: {heating_data.index[0]} to {heating_data.index[-1]}")
else:
    print("❌ No data found for the specified time range")
    raise ValueError("No heating data available for analysis")

In [ ]:
# Data preprocessing for validation
print("🔧 Preprocessing data for validation...")

# Sort data chronologically for proper adaptive learning
heating_data = heating_data.sort_index()

# Filter for heating periods only (when system is active)
heating_active = heating_data[
    (heating_data.get('outlet_temperature', 0) > 20) &  # System is running
    (heating_data['indoor_temperature'] > 15) &  # Valid temperature readings
    (heating_data['indoor_temperature'] < 30) &
    (heating_data['outdoor_temperature'] > -20) &
    (heating_data['outdoor_temperature'] < 25)
].copy()

print(f"📊 Active heating periods: {len(heating_active)} data points")

# Create state transitions for validation
# Each transition represents a heating decision and its outcome
transitions = []

for i in range(len(heating_active) - 1):
    current_state = heating_active.iloc[i]
    next_state = heating_active.iloc[i + 1]
    
    # Skip if time gap is too large (more than 2 hours)
    time_diff = (next_state.name - current_state.name).total_seconds() / 3600
    if time_diff > 2:
        continue
    
    # Create transition record
    transition = {
        'timestamp': current_state.name,
        'current_indoor': current_state['indoor_temperature'],
        'current_outdoor': current_state['outdoor_temperature'],
        'outlet_used': current_state.get('outlet_temperature', 35.0),
        'achieved_indoor': next_state['indoor_temperature'],
        'target_indoor': 21.0,  # Assume standard target
        'pv_power': current_state.get('pv_power', 0),
        'time_diff_hours': time_diff
    }
    
    transitions.append(transition)

transitions_df = pd.DataFrame(transitions)
print(f"✅ Created {len(transitions_df)} state transitions for validation")

if len(transitions_df) == 0:
    print("❌ No valid state transitions found")
    raise ValueError("Insufficient data for validation")

print("\n📊 Transition Overview:")
print(f"   • Average time between states: {transitions_df['time_diff_hours'].mean():.1f} hours")
print(f"   • Indoor temp changes: {transitions_df['achieved_indoor'].mean():.1f}°C ± {transitions_df['achieved_indoor'].std():.1f}°C")
print(f"   • Outlet temperatures used: {transitions_df['outlet_used'].mean():.1f}°C ± {transitions_df['outlet_used'].std():.1f}°C")

In [ ]:
# Initialize models for chronological validation
print("🤖 Initializing models for validation...")

heat_curve = HeatCurveWithReverse()
physics_model = PhysicsModelWithReverse()

# Reset adaptive learning to start fresh
physics_model.reset_adaptive_learning()

print("✅ Models initialized and ready for chronological validation")
print("📈 Starting adaptive learning validation with reverse prediction...")

# Track validation progress
validation_results = {
    'heat_curve_errors': [],
    'physics_errors': [],
    'timestamps': [],
    'better_model': [],  # Track which model performs better at each step
    'physics_learning_metrics': []  # Track learning progression
}

# Process transitions chronologically
print(f"\n🔄 Processing {len(transitions_df)} transitions chronologically...")

for idx, transition in tqdm(transitions_df.iterrows(), total=len(transitions_df), desc="Validating"):
    try:
        # Extract transition data
        timestamp = transition['timestamp']
        achieved_indoor = transition['achieved_indoor']
        target_indoor = transition['target_indoor']
        outdoor_temp = transition['current_outdoor']
        actual_outlet = transition['outlet_used']
        pv_power = transition['pv_power']
        
        # Reverse predict outlet temperatures
        heat_curve_reverse = heat_curve.reverse_predict_outlet_temperature(
            achieved_indoor, target_indoor, outdoor_temp
        )
        
        physics_reverse = physics_model.reverse_predict_outlet_temperature(
            achieved_indoor, target_indoor, outdoor_temp, pv_power=pv_power
        )
        
        # Calculate errors
        heat_curve_error = abs(heat_curve_reverse - actual_outlet)
        physics_error = abs(physics_reverse - actual_outlet)
        
        # Track predictions
        heat_curve.track_reverse_prediction(heat_curve_reverse, actual_outlet, timestamp)
        physics_model.track_reverse_prediction(physics_reverse, actual_outlet, timestamp)
        
        # Update physics model with adaptive learning
        # Use the outlet prediction error as feedback
        physics_model.update_prediction_feedback(
            predicted_temp=achieved_indoor,  # What we predicted indoor would be
            actual_temp=achieved_indoor,     # What it actually was
            outdoor_temp=outdoor_temp,
            outlet_temp=actual_outlet,
            pv_power=pv_power,
            timestamp=timestamp
        )
        
        # Record validation results
        validation_results['heat_curve_errors'].append(heat_curve_error)
        validation_results['physics_errors'].append(physics_error)
        validation_results['timestamps'].append(timestamp)
        validation_results['better_model'].append(
            'Heat Curve' if heat_curve_error < physics_error else 'Physics Model'
        )
        
        # Track learning metrics every 100 predictions
        if len(validation_results['timestamps']) % 100 == 0:
            learning_metrics = physics_model.get_adaptive_learning_metrics()
            validation_results['physics_learning_metrics'].append({
                'prediction_count': len(validation_results['timestamps']),
                'timestamp': timestamp,
                'metrics': learning_metrics
            })
    
    except Exception as e:
        print(f"⚠️ Error processing transition at {timestamp}: {e}")
        continue

print(f"✅ Validation complete! Processed {len(validation_results['timestamps'])} transitions")

In [ ]:
# Analysis: Performance Evolution Over Time
print("📊 PERFORMANCE EVOLUTION ANALYSIS")
print("=" * 50)

# Get final performance metrics
heat_curve_metrics = heat_curve.get_reverse_performance_metrics()
physics_metrics = physics_model.get_reverse_performance_metrics()

# Overall performance comparison
print("\n🏆 FINAL PERFORMANCE COMPARISON:")
print(f"Heat Curve:")
print(f"   • Average outlet error: {heat_curve_metrics['avg_outlet_error']:.2f}°C")
print(f"   • Predictions within 5°C: {heat_curve_metrics['within_5C']:.1f}%")
print(f"   • Total predictions: {heat_curve_metrics['total_predictions']}")

print(f"\nPhysics Model:")
print(f"   • Average outlet error: {physics_metrics['avg_outlet_error']:.2f}°C")
print(f"   • Predictions within 5°C: {physics_metrics['within_5C']:.1f}%")
print(f"   • Total predictions: {physics_metrics['total_predictions']}")

# Determine winner
if heat_curve_metrics['avg_outlet_error'] < physics_metrics['avg_outlet_error']:
    winner = "Heat Curve"
    improvement = physics_metrics['avg_outlet_error'] / heat_curve_metrics['avg_outlet_error']
    print(f"\n🏅 Winner: Heat Curve (performs {improvement:.1f}x better)")
else:
    winner = "Physics Model"
    improvement = heat_curve_metrics['avg_outlet_error'] / physics_metrics['avg_outlet_error']
    print(f"\n🏅 Winner: Physics Model (performs {improvement:.1f}x better)")

# Learning progression analysis
print("\n📈 LEARNING PROGRESSION:")
heat_curve_wins = sum(1 for model in validation_results['better_model'] if model == 'Heat Curve')
physics_wins = sum(1 for model in validation_results['better_model'] if model == 'Physics Model')
total_comparisons = len(validation_results['better_model'])

print(f"   • Heat Curve better: {heat_curve_wins}/{total_comparisons} ({heat_curve_wins/total_comparisons*100:.1f}%)")
print(f"   • Physics Model better: {physics_wins}/{total_comparisons} ({physics_wins/total_comparisons*100:.1f}%)")

# Analyze learning over time (first half vs second half)
mid_point = len(validation_results['better_model']) // 2
first_half = validation_results['better_model'][:mid_point]
second_half = validation_results['better_model'][mid_point:]

first_half_physics_wins = sum(1 for model in first_half if model == 'Physics Model')
second_half_physics_wins = sum(1 for model in second_half if model == 'Physics Model')

print(f"\n🧠 LEARNING ANALYSIS:")
print(f"   • First half: Physics Model won {first_half_physics_wins}/{len(first_half)} ({first_half_physics_wins/len(first_half)*100:.1f}%)")
print(f"   • Second half: Physics Model won {second_half_physics_wins}/{len(second_half)} ({second_half_physics_wins/len(second_half)*100:.1f}%)")

if second_half_physics_wins/len(second_half) > first_half_physics_wins/len(first_half):
    improvement_pct = (second_half_physics_wins/len(second_half) - first_half_physics_wins/len(first_half)) * 100
    print(f"✅ Physics Model improved by {improvement_pct:.1f}% through adaptive learning!")
else:
    decline_pct = (first_half_physics_wins/len(first_half) - second_half_physics_wins/len(second_half)) * 100
    print(f"❌ Physics Model declined by {decline_pct:.1f}% over time")

In [ ]:
# Visualization: Performance Over Time
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# 1. Rolling average outlet prediction errors
window_size = 50
heat_curve_rolling = pd.Series(validation_results['heat_curve_errors']).rolling(window_size).mean()
physics_rolling = pd.Series(validation_results['physics_errors']).rolling(window_size).mean()

ax1.plot(heat_curve_rolling, label='Heat Curve', color='blue', alpha=0.8)
ax1.plot(physics_rolling, label='Physics Model', color='red', alpha=0.8)
ax1.set_xlabel('Prediction Number')
ax1.set_ylabel('Rolling Average Error (°C)')
ax1.set_title(f'Outlet Prediction Error Over Time (Rolling {window_size}-point average)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Cumulative win rate
physics_wins_cumulative = []
cumulative_wins = 0
for i, model in enumerate(validation_results['better_model']):
    if model == 'Physics Model':
        cumulative_wins += 1
    physics_wins_cumulative.append(cumulative_wins / (i + 1) * 100)

ax2.plot(physics_wins_cumulative, color='red', alpha=0.8)
ax2.axhline(y=50, color='gray', linestyle='--', alpha=0.5)
ax2.set_xlabel('Prediction Number')
ax2.set_ylabel('Physics Model Win Rate (%)')
ax2.set_title('Physics Model Performance vs Heat Curve Over Time')
ax2.grid(True, alpha=0.3)
ax2.set_ylim(0, 100)

# 3. Error distribution comparison
bins = np.linspace(0, 20, 30)
ax3.hist(validation_results['heat_curve_errors'], bins=bins, alpha=0.6, label='Heat Curve', color='blue', density=True)
ax3.hist(validation_results['physics_errors'], bins=bins, alpha=0.6, label='Physics Model', color='red', density=True)
ax3.set_xlabel('Outlet Prediction Error (°C)')
ax3.set_ylabel('Probability Density')
ax3.set_title('Error Distribution Comparison')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Performance improvement over time (if data available)
if len(heat_curve_metrics.get('performance_history', [])) > 0 and len(physics_metrics.get('performance_history', [])) > 0:
    hc_history = heat_curve_metrics['performance_history']
    pm_history = physics_metrics['performance_history']
    
    hc_counts = [h['prediction_count'] for h in hc_history]
    hc_errors = [h['avg_error'] for h in hc_history]
    pm_counts = [h['prediction_count'] for h in pm_history]
    pm_errors = [h['avg_error'] for h in pm_history]
    
    ax4.plot(hc_counts, hc_errors, 'o-', label='Heat Curve', color='blue', alpha=0.8)
    ax4.plot(pm_counts, pm_errors, 'o-', label='Physics Model', color='red', alpha=0.8)
    ax4.set_xlabel('Prediction Count')
    ax4.set_ylabel('Average Error (°C)')
    ax4.set_title('Learning Performance Evolution (100-point windows)')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
else:
    ax4.text(0.5, 0.5, 'Insufficient data\nfor learning evolution', 
             transform=ax4.transAxes, ha='center', va='center', fontsize=12)
    ax4.set_title('Learning Performance Evolution')

plt.tight_layout()
plt.show()

# Summary statistics
print("\n📊 VALIDATION SUMMARY:")
print(f"   • Total transitions validated: {len(validation_results['timestamps'])}")
print(f"   • Heat Curve average error: {np.mean(validation_results['heat_curve_errors']):.2f}°C")
print(f"   • Physics Model average error: {np.mean(validation_results['physics_errors']):.2f}°C")
print(f"   • Physics Model final win rate: {physics_wins_cumulative[-1]:.1f}%")

In [ ]:
# Detailed Learning Analysis
print("🧠 DETAILED ADAPTIVE LEARNING ANALYSIS")
print("=" * 50)

# Get detailed learning metrics
final_learning_metrics = physics_model.get_adaptive_learning_metrics()

print("\n📈 Adaptive Learning Status:")
if 'sufficient_data' in final_learning_metrics:
    if final_learning_metrics['sufficient_data']:
        print("✅ Sufficient data for adaptive learning")
        
        if 'calibration_active' in final_learning_metrics:
            print(f"🔄 Calibration active: {final_learning_metrics['calibration_active']}")
        
        if 'learning_phases' in final_learning_metrics:
            phases = final_learning_metrics['learning_phases']
            print(f"📚 Learning phases completed: {len(phases)}")
            
            for i, phase in enumerate(phases[-3:]):  # Show last 3 phases
                print(f"   Phase {len(phases)-2+i}: {phase.get('improvement', 'N/A')}")
    else:
        print("❌ Insufficient data for adaptive learning")
else:
    print("❓ Learning metrics not available")

# Parameter evolution analysis
if len(physics_model.parameter_evolution) > 0:
    print("\n🔧 Parameter Evolution:")
    initial_params = physics_model.parameter_evolution[0]['parameters']
    final_params = physics_model.parameter_evolution[-1]['parameters']
    
    if initial_params and final_params:
        print("   Parameters changed from initial to final values:")
        for param, initial_val in initial_params.items():
            if param in final_params:
                final_val = final_params[param]
                change = ((final_val - initial_val) / initial_val) * 100 if initial_val != 0 else 0
                print(f"      {param}: {initial_val:.3f} → {final_val:.3f} ({change:+.1f}%)")
    else:
        print("   Parameter details not available")
else:
    print("\n❌ No parameter evolution data captured")

# Key insights and recommendations
print("\n💡 KEY INSIGHTS:")

# Performance improvement analysis
if physics_wins_cumulative[-1] > physics_wins_cumulative[len(physics_wins_cumulative)//4]:
    print("✅ Physics model shows learning improvement over time")
    print("   → Adaptive learning is working and improving predictions")
else:
    print("❌ Physics model did not show clear improvement")
    print("   → May need more data or parameter tuning")

# Overall recommendation
if winner == "Physics Model":
    print("\n🏆 RECOMMENDATION: Physics Model with adaptive learning outperforms heat curve")
    print("   → Consider deploying physics-based system for better accuracy")
else:
    print("\n🏆 RECOMMENDATION: Heat curve remains more reliable than physics model")
    print("   → More training data or parameter refinement needed for physics model")

print("\n" + "=" * 50)
print("✅ COMPLETE HISTORICAL VALIDATION FINISHED")
print("📊 All models evaluated with reverse prediction methodology")
print("🧠 Adaptive learning progression tracked")
print("🏆 Performance comparison complete")